In [1]:
import getpass
import os
import tarfile
import json
import time
import torch
import s3fs
import random
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from torchvision.datasets import ImageFolder
from sklearn.manifold import TSNE
from sklearn import metrics
from sklearn.metrics import pairwise_distances_argmin_min


from tqdm import tqdm
from PIL import Image
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split
from data.data_loader import load_cifar10, load_cifar100, load_imagenet, load_prof
from utils.loss_functions import tkd_kdloss
from torch.utils.data import Subset, Dataset, DataLoader
from torchvision.models.resnet import ResNet, BasicBlock, Bottleneck, ResNet18_Weights, ResNet34_Weights

from models_package.models import Teacher, Student

In [2]:
# access_key = getpass.getpass("Enter your access: ")

# secret_key = password = getpass.getpass("Enter your secret: ")

In [3]:
# # Run once to get images onto EC2

# wider_dir = './WIDER'
# if not os.path.exists(wider_dir):
#     os.makedirs(wider_dir)

# # Specify your S3 bucket and file path
# bucket_name = '210bucket'
# s3_file_path = 'wider_attribute_image.tgz'

# # Initialize an S3 filesystem
# s3 = s3fs.S3FileSystem(key=access_key, secret=secret_key)

# # Download the .tgz file from S3
# with s3.open(f"{bucket_name}/{s3_file_path}", 'rb') as s3_file:
#     with tarfile.open(fileobj=s3_file, mode="r:gz") as tar:
#         # Specify the destination directory where you want to store the extracted contents
#         extract_dir = wider_dir # Change this to your desired directory
#         tar.extractall(path=extract_dir)

# print("File downloaded and extracted successfully.")


In [4]:
# # Specify your S3 bucket and directory path
# s3_directory_path = 'wider_attribute_annotation/'

# local_directory = './WIDER/Annotations'  # Change this to your desired directory

# s3_files = s3.ls(f"{bucket_name}/{s3_directory_path}")


# # Create the local directory if it doesn't exist
# os.makedirs(local_directory, exist_ok=True)

# # Download each file from the S3 directory to the local directory
# for s3_file in s3_files:
#     # Get the filename from the S3 file path
#     filename = os.path.basename(s3_file)
    
#     # Download the file to the local directory
#     local_path = os.path.join(local_directory, filename)
#     with s3.open(s3_file, 'rb') as s3_file_obj:
#         with open(local_path, 'wb') as local_file:
#             local_file.write(s3_file_obj.read())

# print("Files downloaded successfully.")


# Load WIDER


In [5]:
# def make_wider(tag, value, data_path):
#     img_path = os.path.join(data_path, "Image")
#     ann_path = os.path.join(data_path, "Annotations")
#     ann_file = os.path.join(ann_path, "wider_attribute_{}.json".format(tag))

#     data = json.load(open(ann_file, "r"))

#     final = []
#     image_list = data['images']
#     for image in image_list:
#         for person in image["targets"]: # iterate over each person
#             tmp = {}
#             tmp['img_path'] = os.path.join(img_path, image['file_name'])
#             tmp['bbox'] = person['bbox']
#             attr = person["attribute"]
#             for i, item in enumerate(attr):
#                 if item == -1:
#                     attr[i] = 0
#                 if item == 0:
#                     attr[i] = value  # pad un-specified samples
#                 if item == 1:
#                     attr[i] = 1
#             tmp["target"] = attr
#             final.append(tmp)

#     json.dump(final, open("data/wider/{}_wider.json".format(tag), "w"))
#     print("data/wider/{}_wider.json".format(tag))

In [6]:
# #run once
# if not os.path.exists("data/wider"):
#     os.makedirs("data/wider")

# # 0 (zero) means negative, we treat un-specified attribute as negative in the trainval set
# make_wider(tag='trainval', value=0, data_path='WIDER') 
# make_wider(tag='test', value=99, data_path='WIDER')

In [7]:
# Hyperparameters
learning_rate = 0.01379 # 0.096779
num_epochs = 2  # 200
num_workers = 2
temperature = 4.0
alpha = 0.9
momentum = 0.9
num_classes = 61
step_size = 30
gamma = 0.1
beta = 0.5

# new parameters
# lr_input = 0.1
# momentum_input = 0.9
weight_decay_input = 5e-4
# epochs = 20
# T = 4.0 # temperatureture
# alpha = 0.9
patience = 5  # for early stopping

batch_size = 256
num_workers = 4

In [8]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [9]:
class DataSet(Dataset):
    def __init__(self,
                ann_files,
                augs,
                img_size,
                dataset,
                ):
        self.dataset = dataset
        self.ann_files = ann_files
        self.augment = self.augs_function(augs, img_size)
        self.transform = transforms.Compose(
            [
                transforms.ToTensor(),
                transforms.Normalize(mean=[0, 0, 0], std=[1, 1, 1])
            ] 
            # In this paper, we normalize the image data to [0, 1]
            # You can also use the so called 'ImageNet' Normalization method
        )
        self.anns = []
        self.load_anns()
        print(self.augment)

        # in wider dataset we use vit models
        # so transformation has been changed
        if self.dataset == "wider":
            self.transform = transforms.Compose(
                [
                    transforms.ToTensor(),
                    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
                ] 
            )        

    def augs_function(self, augs, img_size):            
        t = []
        if 'randomflip' in augs:
            t.append(transforms.RandomHorizontalFlip())
        if 'ColorJitter' in augs:
            t.append(transforms.ColorJitter(brightness=0.5, contrast=0.5, saturation=0.5, hue=0))
        if 'resizedcrop' in augs:
            t.append(transforms.RandomResizedCrop(img_size, scale=(0.7, 1.0)))
        if 'RandAugment' in augs:
            t.append(RandAugment())

        t.append(transforms.Resize((img_size, img_size)))

        return transforms.Compose(t)
    
    def load_anns(self):
        self.anns = []
        for ann_file in self.ann_files:
            json_data = json.load(open(ann_file, "r"))
            self.anns += json_data

    def __len__(self):
        return len(self.anns)

    def __getitem__(self, idx):
        # Make sure the index is within bounds
        idx = idx % len(self)
        ann = self.anns[idx]
        
        try:
            # Attempt to open the image file
            img = Image.open(ann["img_path"]).convert("RGB")

            # If this is the wider dataset, proceed with specific processing
            if self.dataset == "wider":
                x, y, w, h = ann['bbox']
                img_area = img.crop([x, y, x+w, y+h])
                img_area = self.augment(img_area)
                img_area = self.transform(img_area)
                
                # Extract label from image path
                img_path = ann['img_path']
                label = self.extract_label(img_path)  # You might need to implement this method
                
                return {
                    "label": label,
                    "target": torch.Tensor(ann['target']),
                    "img": img_area
                }
            
            # Else, add handling for other dataset types
            else:
                # Handle other dataset types here
                pass

        except Exception as e:
            # If any error occurs during the processing of an image, log the error and the index
            print(f"Error processing image at index {idx}: {e}")
            # Instead of returning None, raise the exception
            raise

    def extract_label(self, img_path):
        # Implement this method based on how you need to extract labels
        # This is just a placeholder implementation
        label = 0
        # Extract the label from the image path
        if "WIDER/Image/train" in img_path:
            label_str = img_path.split("WIDER/Image/train/")[1].split("/")[0]
            label = int(label_str.split("--")[0])
        elif "WIDER/Image/test" in img_path:
            label_str = img_path.split("WIDER/Image/test/")[1].split("/")[0]
            label = int(label_str.split("--")[0])
        return label

In [10]:
train_file = ['data/wider/trainval_wider.json']
test_file = ['data/wider/test_wider.json']


In [11]:
def custom_collate(batch):
    # Filter out any None items in the batch
    batch = [item for item in batch if item is not None]
    # If after filtering the batch is empty, handle this case by either returning an empty tensor or raising an exception
    if len(batch) == 0:
        # Option 1: Return a placeholder tensor (adapt the shape to match your data)
        # return torch.tensor([]), torch.tensor([])
        # Option 2: Raise an exception
        raise ValueError("Batch is empty after filtering out None items.")
    return torch.utils.data.dataloader.default_collate(batch)


In [12]:
train_dataset = DataSet(train_file, augs = ['randomflip'], img_size = 224, dataset = 'wider')
# subset_indices = range(0, 1000)  # Select indices 0 to 999
# train_dataset = Subset(train_dataset, subset_indices)

test_dataset = DataSet(test_file, augs = [], img_size = 224, dataset = 'wider')

for i in range(len(train_dataset)):
    item = train_dataset[i]
    if item is None:
        print(f"None item found at index: {i}")

trainloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers, collate_fn=custom_collate)
testloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers, collate_fn=custom_collate)



Compose(
    RandomHorizontalFlip(p=0.5)
    Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=warn)
)
Compose(
    Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=warn)
)


In [20]:
def print_dataset_details(dataset):
    unique_labels = set()
    for i in range(len(dataset)):
        item = dataset[i]
        unique_labels.add(item['label'])
    print(f"Dataset - Total Samples: {len(dataset)}, Unique Labels: {unique_labels}")

print_dataset_details(train_dataset)

Dataset - Total Samples: 28345, Unique Labels: {0, 1, 3, 4, 6, 7, 11, 15, 17, 18, 19, 20, 22, 25, 27, 28, 30, 31, 33, 35, 36, 37, 39, 43, 44, 50, 51, 54, 57, 58}


In [18]:
import torch
from sklearn.cluster import MiniBatchKMeans
from torch.utils.data import DataLoader, Dataset

# Assuming your dataset is named 'my_dataset'
# Assuming 'img' is a tensor of shape (3, H, W) for each sample

# Function to flatten the image tensors
def flatten_images(images):
    return images.view(images.size(0), -1).numpy()

# Parameters
num_clusters = 20
batch_size = 32  # Adjust as needed

# Create a DataLoader for your original dataset

# Initialize MiniBatchKMeans
kmeans = MiniBatchKMeans(n_clusters=num_clusters, random_state=42)

# Initialize an empty list to store the cluster labels
cluster_labels = []
# Process the dataset in batches
for batch in trainloader:
    img_batch = batch['img']
    flattened_images = flatten_images(img_batch)

    # Partial fit on the batch
    cluster_labels.extend(kmeans.fit_predict(flattened_images))



# Add the cluster labels to the dataset
for i, sample in enumerate(my_dataset):
    sample['cluster_label'] = int(cluster_labels[i])

# Create a new DataLoader based on the clustered data
class ClusteredDataset(Dataset):
    def __init__(self, dataset, cluster_labels):
        self.dataset = dataset
        self.cluster_labels = cluster_labels

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        sample = self.dataset[idx]
        cluster_label = self.cluster_labels[idx]
        sample['cluster_label'] = cluster_label
        return sample

# Create a new dataset instance with clustered labels
clustered_dataset = ClusteredDataset(my_dataset, cluster_labels)

# Create a new dataloader for the clustered dataset
clustered_dataloader = DataLoader(clustered_dataset, batch_size=batch_size, shuffle=True)


/opt/conda/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1934: FutureWarning: The default value of `n_init` will change from 3 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=3)
/opt/conda/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1934: FutureWarning: The default value of `n_init` will change from 3 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=3)
/opt/conda/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1934: FutureWarning: The default value of `n_init` will change from 3 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=3)
/opt/conda/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1934: FutureWarning: The default value of `n_init` will change from 3 to 'auto' in 1.4. Set the value of `n_init` explicitl

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Function to display images and their labels
def show_samples_comparison(original_dataset, clustered_dataset, dataloader, num_samples=5):
    fig, axes = plt.subplots(num_samples, 2, figsize=(8, 2*num_samples))
    fig.tight_layout()

    # Get the original indices from the first batch
    original_indices = [batch['idx'] for batch in dataloader]

    for i in range(num_samples):
        # Original sample
        original_sample = original_dataset[original_indices[i]]
        original_img = original_sample['img'].permute(1, 2, 0).numpy()
        axes[i, 0].imshow(original_img)
        axes[i, 0].set_title(f"Original Label: {original_sample['label']}")

        # Clustered sample
        clustered_sample = clustered_dataset[original_indices[i]]
        clustered_img = clustered_sample['img'].permute(1, 2, 0).numpy()
        axes[i, 1].imshow(clustered_img)
        axes[i, 1].set_title(f"Cluster Label: {clustered_sample['cluster_label']}")

    plt.show()

# Create a new DataLoader for the original dataset to get indices
original_dataloader = DataLoader(my_dataset, batch_size=batch_size, shuffle=False)
original_indices = [batch['idx'] for batch in original_dataloader]

# Set the 'idx' field for each sample in the original dataset
for i, sample in enumerate(my_dataset):
    sample['idx'] = i

# Show samples
show_samples_comparison(my_dataset, clustered_dataset, original_dataloader, num_samples=5)


# Start Training Process

In [ ]:
def _compute_classification_recall(patch_collection, label_field):
    eval_key = "eval_" + label_field
    counts = patch_collection.count_values(eval_key)
    tp, fn = counts.get(True, 0), counts.get(False, 0)
    recall = tp/float(tp + fn) if tp > 0 else 1e-6
    return recall

In [ ]:
def compare_model_size(teacher, student):
    teacher_params = sum(p.numel() for p in teacher.parameters())
    student_params = sum(p.numel() for p in student.parameters())
    return teacher_params, student_params

def compare_inference_time(teacher, student, dataloader):
    dataiter = iter(dataloader)
    data = next(dataiter)
    inputs = data['img']
    
    teacher = teacher.to(device)
    student = student.to(device)
    inputs = inputs.to(device)
    
    start_time = time.time()
    with torch.no_grad():
        teacher_outputs = teacher(inputs)
    teacher_time = time.time() - start_time

    start_time = time.time()
    with torch.no_grad():
        student_outputs = student(inputs)
    student_time = time.time() - start_time
    
    return teacher_time, student_time

def compare_performance_metrics(teacher, student, dataloader):
    teacher.eval()
    student.eval()
    
    all_labels = []
    all_teacher_preds = []
    all_student_preds = []

    for batch in dataloader:
        inputs = batch['img'].to(device)
        labels = batch['label'].to(device)
        
        with torch.no_grad():
            teacher_outputs = teacher(inputs)
            student_outputs = student(inputs)
            
        teacher_preds = torch.argmax(teacher_outputs, dim=1).cpu().numpy()
        student_preds = torch.argmax(student_outputs, dim=1).cpu().numpy()
        
        # Debug: Check if all predictions are the same
        print("Teacher predictions:", np.unique(teacher_preds))
        print("Student predictions:", np.unique(student_preds))
        
        all_labels.append(labels.cpu().numpy())
        all_teacher_preds.append(teacher_preds)
        all_student_preds.append(student_preds)

    all_labels = np.concatenate(all_labels)
    all_teacher_preds = np.concatenate(all_teacher_preds)
    all_student_preds = np.concatenate(all_student_preds)
    
    # Debug: Check final label distribution
    print("Label distribution:", np.unique(all_labels, return_counts=True))
    
    metrics = {
        'accuracy': (accuracy_score(all_labels, all_teacher_preds), accuracy_score(all_labels, all_student_preds)),
        'precision': (precision_score(all_labels, all_teacher_preds, average='weighted', zero_division=0), precision_score(all_labels, all_student_preds, average='weighted', zero_division=0)),
        'recall': (recall_score(all_labels, all_teacher_preds, average='weighted'), recall_score(all_labels, all_student_preds, average='weighted')),
        'f1': (f1_score(all_labels, all_teacher_preds, average='weighted'), f1_score(all_labels, all_student_preds, average='weighted'))
    }

    return metrics

def plot_comparison(labels, teacher_values, student_values, title, ylabel):
    # Convert parameter count to millions
    if 'Parameter Count' in title or 'Parameter Count' in ylabel:
        teacher_values = [value / 1e6 for value in teacher_values]
        student_values = [value / 1e6 for value in student_values]

    x = np.arange(len(labels))  # the label locations
    width = 0.35  # the width of the bars

    fig, ax = plt.subplots()
    rects1 = ax.bar(x - width/2, teacher_values, width, label='Teacher')
    rects2 = ax.bar(x + width/2, student_values, width, label='Student')

    ax.set_ylabel(ylabel)
    ax.set_title(title)
    ax.set_xticks(x)
    ax.set_xticklabels(labels)
    ax.legend()

    fig.tight_layout()

    plt.show()

In [ ]:
# # Load IdenProf dataset
# train_path = '/home/ubuntu/capstone/W210-Capstone/notebooks/idenprof/train'
# test_path = '/home/ubuntu/capstone/W210-Capstone/notebooks/idenprof/test'
# trainloader, testloader  = load_prof(train_path, test_path, batch_size=batch_size)

In [ ]:
# Instantiate the models
###################### Testing 1 ######################
# Create instances of your models
teacher_model = torchvision.models.resnet34(weights=ResNet34_Weights.IMAGENET1K_V1).to(device)
teacher_model.eval()  # Set teacher model to evaluation mode
student_model = torchvision.models.resnet18(weights=None).to(device)

In [ ]:
# Optimizer and scheduler for the student model
optimizer = optim.SGD(student_model.parameters(), lr=learning_rate, momentum=momentum)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=step_size, gamma=gamma)

# Optimizer and scheduler for the teacher model
teacher_optimizer = optim.SGD(teacher_model.parameters(), lr=learning_rate, momentum=momentum)
teacher_scheduler = torch.optim.lr_scheduler.StepLR(teacher_optimizer, step_size=step_size, gamma=gamma)

criterion = nn.CrossEntropyLoss()

In [ ]:
def recall_disparity_loss(outputs, targets, attributes, num_classes):
    """
    Compute the recall disparity loss.

    :param outputs: Tensor of shape (batch_size, num_classes), model's class probabilities or logits.
    :param targets: Tensor of shape (batch_size,), true class indices.
    :param attributes: Tensor of shape (batch_size, num_attributes), binary attributes for each instance.
    :param num_classes: int, number of classes.
    """
    # Ensure we're working with probabilities
    probs = torch.softmax(outputs, dim=1)
    preds = torch.argmax(probs, dim=1)

    # Initialize tensors to store recall for when attributes are present and absent
    recall_when_present = torch.zeros(num_classes, attributes.size(1))
    recall_when_absent = torch.zeros(num_classes, attributes.size(1))

    # Calculate recall for each class based on attributes being present or absent
    for class_idx in range(num_classes):
        for attr_idx in range(attributes.size(1)):
            # Indices of instances with the current class and attribute present/absent
            class_and_attr_present = (targets == class_idx) & (attributes[:, attr_idx] == 1)
            class_and_attr_absent = (targets == class_idx) & (attributes[:, attr_idx] == 0)

            # True positives for current class when attribute is present/absent
            true_positive_present = ((preds == class_idx) & class_and_attr_present).sum().float()
            true_positive_absent = ((preds == class_idx) & class_and_attr_absent).sum().float()

            # Condition positives for current class when attribute is present/absent
            condition_positive_present = class_and_attr_present.sum().float()
            condition_positive_absent = class_and_attr_absent.sum().float()

            # Recall for current class when attribute is present/absent
            recall_when_present[class_idx, attr_idx] = true_positive_present / (condition_positive_present + 1e-8)
            recall_when_absent[class_idx, attr_idx] = true_positive_absent / (condition_positive_absent + 1e-8)

    # Disparity is the absolute difference between recall when attribute is present and when it's absent
    disparity = (recall_when_present - recall_when_absent).abs()

    # Final loss is the mean of disparity across all classes and attributes
    loss = disparity.mean()

    return loss



In [ ]:
# #### finding the optimal learning rate
# def train_teacher(model, trainloader, criterion, optimizer, scheduler, device, num_epochs=5, lr_range=(1e-4, 1e-1), plot_loss=True):
#     model.train()
#     model.to(device)
#     lr_values = np.logspace(np.log10(lr_range[0]), np.log10(lr_range[1]), num_epochs * len(trainloader))  # Generate learning rates for each batch
#     lr_iter = iter(lr_values)
#     losses = []
#     lrs = []
    
#     for epoch in range(num_epochs):
#         for i, (inputs, labels, annotation) in enumerate(tqdm(trainloader)):
#             lr = next(lr_iter)
#             for param_group in optimizer.param_groups:
#                 param_group['lr'] = lr  # Set new learning rate
            
#             inputs, labels = inputs.to(device), labels.to(device)
#             optimizer.zero_grad()
#             outputs = model(inputs)
#             loss = criterion(outputs, labels)
#             loss.backward()
#             optimizer.step()
            
#             losses.append(loss.item())
#             lrs.append(lr)
    
#     # Calculate the derivative of the loss
#     loss_derivative = np.gradient(losses)
    
#     # Find the learning rate corresponding to the minimum derivative (steepest decline)
#     best_lr_index = np.argmin(loss_derivative)
#     best_lr = lrs[best_lr_index]
    
#     if plot_loss:
#         import matplotlib.pyplot as plt
#         plt.figure()
#         plt.plot(lrs, losses)
#         plt.xscale('log')
#         plt.xlabel('Learning Rate')
#         plt.ylabel('Loss')
#         plt.title('Learning Rate Range Test')
#         plt.axvline(x=best_lr, color='red', linestyle='--', label=f'Best LR: {best_lr}')
#         plt.legend()
#         plt.show()
    
#     print(f'Best learning rate: {best_lr}')
#     return best_lr

# ############# input ############## 
# batch_size = 16  #to find the optimal learning rate
# best_lr = train_teacher(teacher_model, trainloader, criterion, teacher_optimizer, teacher_scheduler, device, num_epochs=3)  
# print(best_lr)

In [ ]:
# Function to train the teacher model
def train_teacher(model, trainloader, criterion, optimizer, scheduler, device, num_epochs=1, patience=5):
    model.train()
    model.to(device)
    best_train_loss = float('inf')
    patience_counter = 0

    for epoch in range(num_epochs):
        running_loss = 0.0
        epoch_loss = 0.0  
        num_batches = 0  
        for index, data in enumerate(tqdm(trainloader)):
            inputs = data['img'].to(device)
            labels = data['label'].to(device)
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            epoch_loss += loss.item()
            num_batches += 1
            if index % 100 == 99:  # Print every 100 mini-batches
                print(f"[{epoch + 1}, {i + 1}] loss: {running_loss / 100:.3f}")
                running_loss = 0.0

        epoch_loss /= num_batches  
        
        # Check for early stopping
        if epoch_loss < best_train_loss:
            best_train_loss = epoch_loss
            patience_counter = 0 
            # checkpoint
            torch.save(model.state_dict(), f'teacher_model_weights_ckd_prof_checkpoint.pth')
            torch.save(model, f'teacher_model_ckd_prof_checkpoint.pth')

        else:
            patience_counter += 1

        if patience_counter >= patience:
            print('Early stopping')
            break

        scheduler.step()

    print("Finished Training Teacher")


# Function to train the student model with knowledge distillation
def train_student_with_distillation_disparity(student, teacher, trainloader, criterion, optimizer, scheduler, device, alpha, temperature, num_epochs, patience=5):
    student.train()
    teacher.eval()
    student.to(device)
    teacher.to(device)
    best_train_loss = float('inf')  
    patience_counter = 0 

    for epoch in range(num_epochs):
        running_loss = 0.0 
        epoch_loss = 0.0  
        num_batches = 0  
        epoch_disparity = 0.0
        running_recall = 0.0

        for index, data in enumerate(tqdm(trainloader)):
            inputs = data['img'].to(device)
            labels = data['label'].to(device)
            annot = data['target'].to(device)
            optimizer.zero_grad()
            student_outputs = student(inputs)
            with torch.no_grad():
                teacher_outputs = teacher(inputs)
            
            ce_loss = criterion(student_outputs, labels)
            kd_loss = tkd_kdloss(student_outputs, teacher_outputs, temperature=temperature)  # Make sure this returns a scalar
            recall_difference = recall_disparity_loss(student_outputs, labels, annot, num_classes)
            
            # If not scalar, sum up to make sure the loss is scalar
            if kd_loss.ndim != 0:
                kd_loss = kd_loss.sum()
            if recall_difference.ndim != 0:
                recall_difference = recall_difference.sum()
            
            # Now combine the losses
            loss = alpha * kd_loss + (1 - alpha) * ce_loss + beta * recall_difference
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            epoch_loss += loss.item()
            num_batches += 1
    
            if index % 100 == 99:  
                print(f"[{epoch + 1}, {i + 1}] loss: {running_loss / 100:.3f}")
                running_loss = 0.0
    
        epoch_loss /= num_batches

        # Check for early stopping
        if epoch_loss < best_train_loss:
            best_train_loss = epoch_loss
            patience_counter = 0 
            torch.save(student.state_dict(), f'student_model_weights_ckd_prof_checkpoint.pth')
            torch.save(student, f'student_model_ckd_prof_checkpoint.pth')
        else:
            patience_counter += 1 

        if patience_counter >= patience:
            print('Early stopping')
            break  

        scheduler.step() 

    print("Finished Training Student")

In [ ]:
# Call the function to train the teacher model
train_teacher(teacher_model, trainloader, criterion, teacher_optimizer, teacher_scheduler, device, num_epochs=num_epochs)

# Call the function to train the student model with knowledge distillation
train_student_with_distillation_disparity(student_model, teacher_model, trainloader, criterion, optimizer, scheduler, device, alpha, temperature, num_epochs=num_epochs)


In [ ]:
###################### Testing 1 ######################
# Save the student and teacher model weights and architecture
# torch.save(student_model.state_dict(), 'student_model_weights_ckd_prof.pth')
# torch.save(student_model, 'student_model_ckd_prof.pth')
# print('student weights and architecture saved and exported')

# torch.save(teacher_model.state_dict(), 'teacher_model_weights_ckd_prof.pth')
# torch.save(teacher_model, 'teacher_model_ckd_prof.pth')
# print('teacher weights and architecture saved and exported')

In [ ]:
# ###################### Testing 2 ######################
# # Save the student and teacher model weights and architecture
# torch.save(student_model.state_dict(), 'student_model_weights_ckd_2.pth')
# torch.save(student_model, 'student_model_ckd_2.pth')
# print('weights and architecture saved and exported')

# torch.save(teacher_model.state_dict(), 'teacher_model_weights_ckd_2.pth')
# torch.save(teacher_model, 'teacher_model_ckd_2.pth')
# print('teacher weights and architecture saved and exported')

In [ ]:
# Call the comparison and plotting functions after training
teacher_params, student_params = compare_model_size(teacher_model, student_model)
teacher_time, student_time = compare_inference_time(teacher_model, student_model, trainloader)
performance_metrics = compare_performance_metrics(teacher_model, student_model, trainloader)

# Extracting the metric values for plotting
performance_labels = ['accuracy', 'precision', 'recall', 'f1']
teacher_performance_values = [performance_metrics[metric][0] for metric in performance_labels]
student_performance_values = [performance_metrics[metric][1] for metric in performance_labels]

# Plotting the comparison for performance metrics
plot_comparison(performance_labels, teacher_performance_values, student_performance_values, 'Performance Comparison', 'Score')

# Plotting the comparison for model size
model_size_labels = ['Model Size']
teacher_model_size_values = [teacher_params]
student_model_size_values = [student_params]
plot_comparison(model_size_labels, teacher_model_size_values, student_model_size_values, 'Model Size Comparison', 'Parameter Count (millions)')

# Plotting the comparison for inference time
inference_time_labels = ['Inference Time']
teacher_inference_time_values = [teacher_time]
student_inference_time_values = [student_time]
plot_comparison(inference_time_labels, teacher_inference_time_values, student_inference_time_values, 'Inference Time Comparison', 'Time (s)')

In [ ]:
performance_metrics

In [ ]:
# teacher_model = torchvision.models.resnet34(weights=None)

# weights_path = 'teacher_model_weights_ckd_prof.pth'

# teacher_model.load_state_dict(torch.load(weights_path))

# student_model = torchvision.models.resnet18(weights=None)

# weights_path = 'student_model_weights_ckd_prof.pth'

# student_model.load_state_dict(torch.load(weights_path))

# Call the comparison and plotting functions after training
teacher_params, student_params = compare_model_size(teacher_model, student_model)
teacher_time, student_time = compare_inference_time(teacher_model, student_model, testloader)
performance_metrics = compare_performance_metrics(teacher_model, student_model, testloader)

# Extracting the metric values for plotting
performance_labels = ['accuracy', 'precision', 'recall', 'f1']
teacher_performance_values = [performance_metrics[metric][0] for metric in performance_labels]
student_performance_values = [performance_metrics[metric][1] for metric in performance_labels]

# Plotting the comparison for performance metrics
plot_comparison(performance_labels, teacher_performance_values, student_performance_values, 'Performance Comparison', 'Score')

# Plotting the comparison for model size
model_size_labels = ['Model Size']
teacher_model_size_values = [teacher_params]
student_model_size_values = [student_params]
plot_comparison(model_size_labels, teacher_model_size_values, student_model_size_values, 'Model Size Comparison', 'Parameter Count (millions)')

# Plotting the comparison for inference time
inference_time_labels = ['Inference Time']
teacher_inference_time_values = [teacher_time]
student_inference_time_values = [student_time]
plot_comparison(inference_time_labels, teacher_inference_time_values, student_inference_time_values, 'Inference Time Comparison', 'Time (s)')

In [ ]:
def calculate_recall(preds, targets, condition):
    """
    Calculate recall for a given condition in a multi-class setting.

    :param preds: Predicted classes.
    :param targets: True classes.
    :param condition: Boolean tensor indicating the condition (subset) for which to calculate recall.
    :return: Recall value.
    """
    if condition.sum() == 0:  # No samples meet the condition
        return 0.0

    filtered_preds = preds[condition]
    filtered_targets = targets[condition]

    true_positive = (filtered_preds == filtered_targets).sum().float()
    condition_positive = filtered_targets.size(0)

    recall = true_positive / condition_positive if condition_positive > 0 else 0.0
    return recall


def evaluate_disparity(model, dataloader, num_classes):
    """
    Evaluate the disparity on the test data.

    :param model: The trained model.
    :param dataloader: DataLoader for the test data.
    :param num_classes: The number of classes in the dataset.
    :return: Average disparity across all attributes and classes.
    """
    model.eval()
    total_disparity = 0.0
    num_batches = 0

    with torch.no_grad():
        for batch in dataloader:
            inputs = batch['img'].to(device)
            targets = batch['label'].to(device)
            attributes = batch['target'].to(device)

            outputs = model(inputs)
            preds = torch.argmax(outputs, dim=1)

            batch_disparity = 0.0
            for class_idx in range(num_classes):
                for attr_idx in range(attributes.size(1)):
                    condition_present = (attributes[:, attr_idx] == 1) & (targets == class_idx)
                    condition_absent = (attributes[:, attr_idx] == 0) & (targets == class_idx)

                    recall_present = calculate_recall(preds, targets, condition_present)
                    recall_absent = calculate_recall(preds, targets, condition_absent)

                    disparity = abs(recall_present - recall_absent)
                    batch_disparity += disparity

                    # Debug information
                    print(f"Class: {class_idx}, Attr: {attr_idx}, Disparity: {disparity}")

            batch_disparity /= (num_classes * attributes.size(1))  # Normalize by number of classes and attributes
            total_disparity += batch_disparity
            num_batches += 1

    average_disparity = total_disparity / num_batches
    return average_disparity


In [ ]:
# Example usage:
disparity = evaluate_disparity(student_model, testloader, num_classes=num_classes)
print(f'Average recall disparity across all attributes and classes: {disparity}')
